In this notebook, we 

In [ ]:
import os
import sys
from pathlib import Path

current_dir = Path(os.getcwd())
project_root = current_dir.parent.parent
sys.path.append(str(project_root))

import sparse_qubo  # noqa: E402
from examples.common import calculate_chain_break_rate, sampling_with_dwave  # noqa: E402
from examples.shift_scheduling.problem import (  # noqa: E402
    check_feasibility,
    check_incompatible_pairs,
    create_incompatible_pairs,
    create_scheduling_cost_matrix,
    create_scheduling_problem_bqm,
)

In [24]:
num_days = 6
num_workers = 2 * num_days
num_incompatible_pairs = 5
shifts_per_day = 2
shifts_per_worker = 1
naive_penalty_factor = 20
dc_penalty_factor = 15
seed = 42

In [25]:
incompatible_pairs = create_incompatible_pairs(num_workers, num_incompatible_pairs, seed=seed)
cost_matrix = create_scheduling_cost_matrix(num_days, num_workers, seed=seed)

print(incompatible_pairs)
print(cost_matrix)

{(0, 1), (3, 4), (2, 7), (6, 10), (8, 11)}
[[-2. -1. -3. -1. -1. -4. -3. -3. -3. -1. -2. -3.]
 [-1. -4. -2. -4. -2. -1. -5. -2. -4. -1. -2. -5.]
 [-5. -3. -3. -4. -2. -2. -3. -2. -2. -5. -3. -1.]
 [-3. -1. -5. -4. -2. -5. -2. -4. -4. -5. -4. -1.]
 [-4. -2. -2. -2. -2. -1. -3. -5. -2. -4. -2. -4.]
 [-4. -2. -1. -4. -4. -2. -4. -4. -2. -2. -5. -1.]]


In [26]:
naive_bqm = create_scheduling_problem_bqm(
    sparse_qubo.NetworkType.NAIVE,
    num_days,
    num_workers,
    shifts_per_day,
    shifts_per_worker,
    incompatible_pairs,
    cost_matrix,
    penalty_factor=naive_penalty_factor,
)
dc_bqm = create_scheduling_problem_bqm(
    sparse_qubo.NetworkType.DIVIDE_AND_CONQUER,
    num_days,
    num_workers,
    shifts_per_day,
    shifts_per_worker,
    incompatible_pairs,
    cost_matrix,
    penalty_factor=dc_penalty_factor,
)

In [27]:
n_inter = naive_bqm.num_interactions
dc_inter = dc_bqm.num_interactions
reduction = (1 - dc_inter / n_inter) * 100

print(f"{'Metric':<20} | {'Naive':<10} | {'D&C':<10}")
print("-" * 46)
print(f"{'Num Variables':<20} | {len(naive_bqm.variables):<10} | {len(dc_bqm.variables):<10}")
print(f"{'Num Interactions':<20} | {naive_bqm.num_interactions:<10} | {dc_bqm.num_interactions:<10}")
print(f"Interactions Reduced: {reduction:.1f}% ({n_inter} -> {dc_inter})")

Metric               | Naive      | D&C       
----------------------------------------------
Num Variables        | 72         | 720       
Num Interactions     | 576        | 558       
Interactions Reduced: 3.1% (576 -> 558)


In [28]:
import dwave.cloud
import dwave.system

In [29]:
client = dwave.cloud.Client.from_config()
print(client.get_solvers())

[BQMSolver(name='hybrid_binary_quadratic_model_version2p'), DQMSolver(name='hybrid_discrete_quadratic_model_version1p'), CQMSolver(name='hybrid_constrained_quadratic_model_version1p'), NLSolver(name='hybrid_nonlinear_program_version1p'), StructuredSolver(name='Advantage_system6.4', graph_id='01dae5a273'), StructuredSolver(name='Advantage_system4.1', graph_id='01d07086e1'), StructuredSolver(name='Advantage2_system1.11', graph_id='01dceba9f7')]


In [30]:
solver_name = "Advantage2_system1.11"
num_reads = 1000
annealing_time = 20
naive_chain_strength = None
dc_chain_strength = None

In [31]:
naive_result = sampling_with_dwave(
    solver_name,
    naive_bqm,
    num_reads=num_reads,
    annealing_time=annealing_time,
    chain_strength=naive_chain_strength,
)

In [32]:
dc_result = sampling_with_dwave(
    solver_name,
    dc_bqm,
    num_reads=num_reads,
    annealing_time=annealing_time,
    chain_strength=dc_chain_strength,
)

In [33]:
naive_embedding = sum([len(v) for v in naive_result.info["embedding_context"]["embedding"].values()])
naive_chain_strength = naive_result.info["embedding_context"]["chain_strength"]
dc_embedding = sum([len(v) for v in dc_result.info["embedding_context"]["embedding"].values()])
dc_chain_strength = dc_result.info["embedding_context"]["chain_strength"]


naive_first = naive_result.first.sample
dc_first = dc_result.first.sample

is_constrained_naive = check_feasibility(naive_first, num_days, num_workers, shifts_per_day, shifts_per_worker)
is_compatible_naive = check_incompatible_pairs(naive_first, num_days, num_workers, incompatible_pairs)
is_constrained_dc = check_feasibility(dc_first, num_days, num_workers, shifts_per_day, shifts_per_worker)
is_compatible_dc = check_incompatible_pairs(dc_first, num_days, num_workers, incompatible_pairs)

naive_df = naive_result.to_pandas_dataframe()
dc_df = dc_result.to_pandas_dataframe()

naive_energy_average = naive_df["energy"].mean()
dc_energy_average = dc_df["energy"].mean()


print(f"{'Metric':<20} | {'Naive':<10} | {'D&C':<10}")
print("-" * 46)
print(f"{'Feasible':<20} | {is_constrained_naive!s:<10} | {is_constrained_dc!s:<10}")
print(f"{'Compatible':<20} | {is_compatible_naive!s:<10} | {is_compatible_dc!s:<10}")
print(f"{'Chain Strength':<20} | {f'{naive_chain_strength:.3f}':<10} | {f'{dc_chain_strength:.3f}':<10}")
print(f"{'Embedding Qubits':<20} | {naive_embedding:<10} | {dc_embedding:<10}")
print(
    f"{'Chain Break Rate':<20} | {f'{calculate_chain_break_rate(naive_df):.3f}':<10} | {f'{calculate_chain_break_rate(dc_df):.3f}':<10}"
)
print(f"{'Best Solution Energy':<20} | {naive_result.first.energy:<10} | {dc_result.first.energy:<10}")
print(f"{'Average Energy':<20} | {f'{naive_energy_average:.3f}':<10} | {f'{dc_energy_average:.3f}':<10}")

Metric               | Naive      | D&C       
----------------------------------------------
Feasible             | True       | True      
Compatible           | True       | True      
Chain Strength       | 58.372     | 12.934    
Embedding Qubits     | 348        | 951       
Chain Break Rate     | 0.014      | 0.004     
Best Solution Energy | -33.0      | -47.0     
Average Energy       | 77.399     | 29.578    
